In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
182,ABW,North America,Aruba,2020-09-16,3152.0,92.0,80.429,22.0,2.0,1.000,...,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
443,AFG,Asia,Afghanistan,2020-09-16,38855.0,40.0,47.857,1436.0,10.0,2.571,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
622,AGO,Africa,Angola,2020-09-16,3439.0,51.0,65.429,134.0,0.0,2.000,...,5819.495,NaN,276.045,3.94,NaN,NaN,26.664,NaN,61.15,0.581


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-15,France,393911.0,0.0,30974.0,0.0,0.0,0.0
0,2020-09-15,China,90246.0,0.0,4740.0,0.0,0.0,0.0
0,2020-09-15,Italy,290086.0,0.0,35649.0,0.0,0.0,0.0
0,2020-09-15,Spain,601194.0,0.0,29865.0,0.0,0.0,0.0
0,2020-09-15,United States,6634963.0,0.0,195511.0,0.0,0.0,0.0
0,2020-09-15,World,29863369.0,0.0,938412.0,0.0,0.0,0.0
0,2020-09-15,United Kingdom,373834.0,0.0,41674.0,0.0,0.0,0.0
0,2020-09-15,Germany,262659.0,0.0,9368.0,0.0,0.0,0.0
0,2020-09-15,Iran,408174.0,0.0,23531.0,0.0,0.0,0.0
0,2020-09-15,Turkey,294087.0,0.0,7164.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-16,France,393911.0,395104.0,30974.0,30999.0,-1193.0,-25.0
0,2020-09-16,China,90246.0,90243.0,4740.0,4736.0,3.0,4.0
0,2020-09-16,Italy,290086.0,289990.0,35649.0,35633.0,96.0,16.0
0,2020-09-15,Spain,601194.0,603167.0,29865.0,30004.0,-1973.0,-139.0
0,2020-09-16,United States,6634963.0,6606293.0,195511.0,195937.0,28670.0,-426.0
0,2020-09-16,World,29863369.0,29611395.0,938412.0,935767.0,251974.0,2645.0
0,2020-09-16,United Kingdom,373834.0,374228.0,41674.0,41664.0,-394.0,10.0
0,2020-09-16,Germany,262659.0,263663.0,9368.0,9368.0,-1004.0,0.0
0,2020-09-16,Iran,408174.0,407353.0,23531.0,23453.0,821.0,78.0
0,2020-09-16,Turkey,294087.0,294620.0,7164.0,7186.0,-533.0,-22.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")